<a href="https://colab.research.google.com/github/intanelaqsha/intanelaqsha/blob/main/Data_Analyst_Task_MainGames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data= pd.read_csv('clips.csv')


### TASK NUMBER 1

In [ ]:
# Convert 'created_at' column to datetime type
data['year_month'] = pd.to_datetime(data['created_at'])

# Set 'year_month' column as the DataFrame index
data.set_index('year_month', inplace=True)

# Resample data with monthly frequency
monthly_data = data.resample('M').agg({
    'created_at': 'count',          # Total clips
    'user_id': 'nunique',           # Total users (unique value)
    'gamesession_Id': 'nunique'     # Total processed gamesession (unique value)
})

# Rename columns for clarity
monthly_data.columns = ['Number of Clips', 'Number of Users', 'Number of Gamesession']

# Convert the index to 'YYYY-MM' format (year-month) without the day
monthly_data.index = monthly_data.index.strftime('%Y-%m')

# Add 'year_month' column back to the DataFrame
monthly_data['year_month'] = monthly_data.index

# Reorder columns with 'year_month' as the first column
monthly_data = monthly_data[['year_month', 'Number of Clips', 'Number of Users', 'Number of Gamesession']]

# Display the result
print("Clip Information, User, and Game Session per Month:")
print(monthly_data)

# Save the DataFrame to Excel with 'year_month' included
monthly_data.to_excel('TASK1.xlsx', index=False)


Clip Information, User, and Game Session per Month:
           year_month  Number of Clips  Number of Users  Number of Gamesession
year_month                                                                    
2023-06       2023-06            81789             1093                   6080
2023-07       2023-07            77776              978                   5518
2023-08       2023-08            80231             1067                   5354
2023-09       2023-09            87737             1210                   6152
2023-10       2023-10            88862             1316                   5830
2023-11       2023-11            82608             1199                   4810
2023-12       2023-12            53028             1083                   3101


### TASK 2

In [ ]:

data['created_at'] = pd.to_datetime(data['created_at'])

# Filter data for the last 3 months
end_date = data['created_at'].max()
start_date = end_date - pd.DateOffset(weeks=12)
filtered_data = data.loc[(data['created_at'] >= start_date) & (data['created_at'] <= end_date)].copy()

# Create a new 'week' column to determine the week from the 'created_at' date
filtered_data['week'] = filtered_data['created_at'].dt.to_period('W')

# Calculate the number of clips for each clip type each week
weekly_clips_count = filtered_data.groupby(['week', 'clip_id']).size().unstack(fill_value=0)

# Display the result
print("Weekly Total Clips of Each Clip Type (Last 3 Months): \n", weekly_clips_count)

# Add 'week' column back to the DataFrame
weekly_clips_count['week'] = weekly_clips_count.index

# Reorder columns with 'week' as the first column
weekly_clips_count = weekly_clips_count[['week'] + list(weekly_clips_count.columns[:-1])]

# Save the DataFrame to Excel with 'week' included
weekly_clips_count.to_excel('TASK2.xlsx', index=False)


Weekly Total Clips of Each Clip Type (Last 3 Months): 
 clip_id                -1      1     2   3    5   6   7     8
week                                                         
2023-09-18/2023-09-24   0   1990    59   1    0   3   0     0
2023-09-25/2023-10-01   0  22461   759  40  274  16   0     0
2023-10-02/2023-10-08   0  21453   747  32  154   6   0     0
2023-10-09/2023-10-15   0  21238   712  43  166  15   0     0
2023-10-16/2023-10-22   0  18973   565  55  175  12   1    66
2023-10-23/2023-10-29   0  15279   601  38  221   7   1   384
2023-10-30/2023-11-05   0  14386   983  21  219   8   0   589
2023-11-06/2023-11-12   0  15964   798  45  180  10   0   370
2023-11-13/2023-11-19   0  20328   893  35  199  10   0   450
2023-11-20/2023-11-26   0  19517  1018  34  154  13   3   238
2023-11-27/2023-12-03   0  18524   829  24  168   5   1   313
2023-12-04/2023-12-10   0  21426   767  55  239  16   3  2205
2023-12-11/2023-12-17   1  17541   604  20  521   4   0   516


### TASK 3

In [ ]:
#OPSIONAL
# Menghitung jumlah jenis permainan dalam data
jumlah_jenis_permainan = data['game_name'].nunique()

# Menampilkan hasil
print("Jumlah Jenis Permainan dalam Data:", jumlah_jenis_permainan)


Jumlah Jenis Permainan dalam Data: 56


In [ ]:
# Group data by 'game_name' and 'event_name', then get the top 3 moments and their average duration
top_moments = data.groupby(['game_name', 'event_name']).agg(
    top_3_moments=('id', 'count'),  # Get the count of clips for each moment (top 3)
    avg_duration=('duration', 'mean')  # Get the average duration for each moment
).sort_values(by=['game_name', 'top_3_moments'], ascending=[True, False]).groupby('game_name').head(3)

# Display the top 3 clip moments for each game and their average duration
print("Top 3 Clip Moments and Average Duration for Each Game:")
print(top_moments)

# determining the name of the file
file_name = 'Top 3 Moments.xlsx'

# saving the excel
top_moments.to_excel(file_name)

Top 3 Clip Moments and Average Duration for Each Game:
                                 top_3_moments  avg_duration
game_name    event_name                                     
Among Us     completing task               165     29.000000
             defeat                         62     28.483871
             High Viewer                    10     59.000000
Apex Legends Single kill                 18527     28.072867
             Chain kill                   3965     39.821690
...                                        ...           ...
chatting     emotional-moment             3453     53.276281
             voice-command                  47     79.574468
starfield    mission-updated               616     29.000000
             mission-rewards                42     29.000000
             epic-item-acquired             26     29.000000

[133 rows x 2 columns]


In [ ]:
# Replace missing values (NaN) in the 'event_name' column with an empty string
data['event_name'] = data['event_name'].fillna('')

# Create a new column 'contains_kill' to indicate if 'kill' is present in 'event_name'
data['contains_kill'] = data['event_name'].str.contains('kill', case=False)

# To calculate the total number of clips containing 'kill' in the event_name
total_kills = data[data['contains_kill']]['created_at'].count()

kill = data['contains_kill'],data['event_name']

print("Total number of clips generated for moment that contain 'kill':", total_kills)

# Create a DataFrame from the tuple
kill_df = pd.DataFrame({'contains_kill': kill[0], 'event_name': kill[1]})

# Save the DataFrame to Excel
kill_df.to_excel('kill_data.xlsx', index=False)




Total number of clips generated for moment that contain 'kill': 299175


Exception ignored in: <function ZipFile.__del__ at 0x78e65e72e200>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
